# Generating eval dataset

In this notebook i export dev and test dataset in the format:

<code> qid | query | docid | doc </code>

To make the evaluation of our document ranking model.

In [5]:
# Downloading data
! wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docdev-queries.tsv.gz
! wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docdev-top100.gz
! wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docdev-qrels.tsv.gz
    
! wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz
! wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-doctest2019-top100.gz
! wget https://trec.nist.gov/data/deep/2019qrels-docs.txt

--2020-06-07 00:11:11--  https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docdev-queries.tsv.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 40.112.152.16
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|40.112.152.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 91837 (90K) [application/x-gzip]
Saving to: ‘msmarco-docdev-queries.tsv.gz’

msmarco-docdev-quer 100%[===================>]  89.68K   261KB/s    in 0.3s    

2020-06-07 00:11:13 (261 KB/s) - ‘msmarco-docdev-queries.tsv.gz’ saved [91837/91837]

--2020-06-07 00:11:13--  https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docdev-top100.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 40.112.152.16
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|40.112.152.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5701839 (5.4M) [application/x-gzip]
Sav

In [8]:
import gzip
import random
import time
import csv

# Lets store dev and test queries

In [17]:
# Vamos armazenar 
path = "msmarco-docdev-queries.tsv.gz"

dev_queries =  {}
len_q = []

for line in gzip.open(path, mode='rt'):

  # items será um vetor onde o primeiro elemento [0] será o qid e o [1] a string de busca
  items = line.strip().split('\t')

  # Adicionamos no vetor de queries cada uma das strings de busca
  dev_queries[items[0]] = items[1]

  # Adicionamos a quantidade de tokens da respectiva query
  len_q.append(len(items[1].split()))

In [18]:
print(max(len_q), len(len_q))

28 5193


In [19]:
# Vamos armazenar 
path = "msmarco-test2019-queries.tsv.gz"

test_queries =  {}
len_q = []

for line in gzip.open(path, mode='rt'):

  # items será um vetor onde o primeiro elemento [0] será o qid e o [1] a string de busca
  items = line.strip().split('\t')

  # Adicionamos no vetor de queries cada uma das strings de busca
  test_queries[items[0]] = items[1]

  # Adicionamos a quantidade de tokens da respectiva query
  len_q.append(len(items[1].split()))

In [20]:
print(max(len_q), len(len_q))

13 200


# Now lets store top100 test and dev documents

In [22]:
# Reading top 100 
import csv 
dev_top100 = {}
with gzip.open("msmarco-docdev-top100.gz", 'rt', encoding='utf8') as top100f:
    
    for line in top100f:
        [topicid, _, unjudged_docid, rank, _, _] = line.split()
        
        if topicid in dev_top100:
            dev_top100[topicid].append(unjudged_docid)
        else:
            dev_top100[topicid] = [unjudged_docid]

In [23]:
print(len(dev_top100))

5193


In [25]:
# Reading top 100 
import csv 
test_top100 = {}
with gzip.open("msmarco-doctest2019-top100.gz", 'rt', encoding='utf8') as top100f:
    
    for line in top100f:
        [topicid, _, unjudged_docid, rank, _, _] = line.split()
        
        if topicid in test_top100:
            test_top100[topicid].append(unjudged_docid)
        else:
            test_top100[topicid] = [unjudged_docid]

In [26]:
print(len(test_top100))

200


# Storing qrels in dev and test

In [45]:
dev_qrel = {}

with gzip.open("msmarco-docdev-qrels.tsv.gz", 'rt', encoding='utf8') as f:
    tsvreader = csv.reader(f, delimiter=" ")
    for [topicid, _, docid, rel] in tsvreader:
        if(rel == '1'):
            if topicid in dev_qrel:
                dev_qrel[topicid].append(docid)
            else:
                dev_qrel[topicid] = [docid]

In [48]:
for v in dev_qrel.values():
    if(len(v) > 1):
        print(len(v))

In [44]:
test_qrel = {}

with open("2019qrels-docs.txt", 'rt', encoding='utf8') as f:
    tsvreader = csv.reader(f, delimiter=" ")
    for [topicid, _, docid, rel] in tsvreader:
        if(rel == '1'):
            if topicid in test_qrel:
                test_qrel[topicid].append(docid)
            else:
                test_qrel[topicid] = [docid]

In [49]:
for v in test_qrel.values():
    if(len(v) > 1):
        print(len(v))

41
628
142
109
36
28
21
12
181
118
15
48
28
133
145
33
147
171
358
32
35
33
136
3
45
2
40
240
32
223
26
235
64
51
335
266
51
248
28
14
73


In [51]:
# Documents

init_time = time.time()
# Vamos armazenar 
path = "msmarco-docs.tsv.gz"

docs = dict()
# length body
len_b = []
# length title
len_t = []
# length total
len_total = []

n_fail = 0 # Quantidade de arquivos sem 4 elementos
i = 0
for line in gzip.open(path, mode='rt'):

  # items será um vetor onde o segundo elemento [2] será a string do title e o [3] a string do body
    items = line.strip().split('\t')


    # Se items não tiver shape [1,4] então não armazenamos nada (tem casos que não tem título e nem body)
    if(len(items) == 4):
        total = items[2] + ' ' + items[3]

      # Adicionamos no vetor de queries cada uma das strings de busca
        docs[items[0]] = total

      # Adicionamos a quantidade de tokens da respectiva query
        len_b.append(len(items[3].split()))
        len_t.append(len(items[2].split()))
        len_total.append(len(total.split()))

    else:
        n_fail += 1

end_time = time.time()

print(f"A célula demorou {end_time - init_time}s para rodar. O código detectou {n_fail} linhas sem 4 itens.")

A célula demorou 975.1222608089447s para rodar. O código detectou 12014 linhas sem 4 itens.


# So now we need to create the dev and test dataset that we we will run and rank to evaluate

To this we need files in the format: 

<code> qid | query | docid | document </code>

Where qid will be the same every 100 steps

In [52]:
list(dev_queries.keys())[0]

'174249'

In [54]:
dev_top100['174249']

['D3126539',
 'D978773',
 'D399803',
 'D2204704',
 'D3126541',
 'D398816',
 'D2168983',
 'D3126537',
 'D3297846',
 'D531991',
 'D2479861',
 'D3281599',
 'D627773',
 'D2012180',
 'D1318897',
 'D1801824',
 'D808582',
 'D3126538',
 'D873580',
 'D2204703',
 'D1005285',
 'D2763949',
 'D1758439',
 'D115272',
 'D884651',
 'D931100',
 'D3074165',
 'D282707',
 'D2115286',
 'D3126536',
 'D1877918',
 'D643861',
 'D2286484',
 'D2369488',
 'D810917',
 'D3126540',
 'D2054522',
 'D2204701',
 'D931102',
 'D2867946',
 'D2774813',
 'D1180095',
 'D747478',
 'D2054518',
 'D1820311',
 'D3343476',
 'D2260493',
 'D36404',
 'D903538',
 'D3143578',
 'D1859054',
 'D2003228',
 'D2282690',
 'D3053559',
 'D2102549',
 'D2015845',
 'D1632207',
 'D3143575',
 'D809059',
 'D2544812',
 'D3189144',
 'D3144156',
 'D1124473',
 'D1227017',
 'D655257',
 'D1809352',
 'D2869616',
 'D1407030',
 'D3144154',
 'D2241255',
 'D3066281',
 'D2210293',
 'D2749624',
 'D3209412',
 'D3114612',
 'D3024420',
 'D954842',
 'D2736584',
 'D2553

In [61]:
def generate_eval_df(outfile, df = 'dev', gen = True):
    
    debug_results = []
    
    with open(outfile, 'w', encoding="utf8") as out:
        
        errors = 0
        
        if(df == 'dev'):
            top100 = dev_top100
            queries = dev_queries
        else:
            top100 = test_top100
            queries = test_queries
        
        qids = list(queries.keys())
        

        for q in qids:
            try:
                for docid in top100[q]:
                    try:
                        
                        if(gen):
                            debug_results.append([q, queries[q], docid, docs[docid]])
                        else:
                            out.write(q + "\t" + queries[q] + "\t" + docid + "\t" + docs[docid] + "\n")
                        
                    except:
                        errors += 1
                        continue
            except:
                errors += 1
                continue
        
    return debug_results, errors

# Generating eval datasets

In [72]:
# DEV EVAL
debugs, errors = generate_eval_df('dev_eval.tsv', df = 'dev', gen = False)
print(f"We miss {errors} observations by some error")

We miss 17 observations by some error


In [73]:
# TEST EVAL
debugs, errors = generate_eval_df('test_eval.tsv', df = 'test', gen = False)
print(f"We miss {errors} observations by some error")

We miss 1 observations by some error


# Checking the format of these files

In [79]:
max_lines = 10
lines = 0
with open("test_eval.tsv", 'rt', encoding='utf8') as f:
    tsvreader = csv.reader(f, delimiter="\t")
    for [qid, query, docid, doc] in tsvreader:
        print(qid, query, docid, doc[:20])
        
        lines += 1 
        
        if(lines>max_lines):
            break

1108939 what slows down the flow of blood D388799 . January, 2013Blood
1108939 what slows down the flow of blood D290852 What controls the fl
1108939 what slows down the flow of blood D2449334 How does ice affect 
1108939 what slows down the flow of blood D884740 The Circulatory syst
1108939 what slows down the flow of blood D2449337 6 How does the remov
1108939 what slows down the flow of blood D1680183 A & P Lecture #4 Stu
1108939 what slows down the flow of blood D1107957 Chapter 19, blood ve
1108939 what slows down the flow of blood D2772087 Constant flow of blo
1108939 what slows down the flow of blood D2143811 Chapter 21: Blood ve
1108939 what slows down the flow of blood D290854 Constant flow of blo
1108939 what slows down the flow of blood D3486983 Lab7.6 "67 terms chr


In [80]:
max_lines = 10
lines = 0
with open("dev_eval.tsv", 'rt', encoding='utf8') as f:
    tsvreader = csv.reader(f, delimiter="\t")
    for [qid, query, docid, doc] in tsvreader:
        print(qid, query, docid, doc[:20])
        
        lines += 1 
        
        if(lines>max_lines):
            break

174249 does xpress bet charge to deposit money in your account D3126539 BetYetu- Registratio
174249 does xpress bet charge to deposit money in your account D978773 Insight Prepaid Card
174249 does xpress bet charge to deposit money in your account D399803 Is online gambling l
174249 does xpress bet charge to deposit money in your account D2204704 Online Betting Sites
174249 does xpress bet charge to deposit money in your account D3126541 Elitebet Kenya Paybi
174249 does xpress bet charge to deposit money in your account D398816 . Easy Pay Xpress Pl
174249 does xpress bet charge to deposit money in your account D2168983 US Greyhound Racing 
174249 does xpress bet charge to deposit money in your account D3126537 SPORTS BETTING SITES
174249 does xpress bet charge to deposit money in your account D3297846 DFW Airport Parking 
174249 does xpress bet charge to deposit money in your account D531991 Does Charles Schwab 
174249 does xpress bet charge to deposit money in your account D2479861 Co